<a href="https://colab.research.google.com/github/jepennerhahn/X-rayEmission/blob/main/X_ray_emission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This notebook will use xraylib to calculate sample properties

In [2]:
#@title First we need to install a few things. 

#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left.  This will take a few minutes.
%%capture

'''Copyright (c) 2022 J.E. Penner-Hahn

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.'''

import os
#Load a bash file that will install miniconda and xraylib

if not os.path.isdir('X-ray_Absorbance/'):
    !git clone "https://github.com/jepennerhahn/X-ray_Absorbance.git"

import subprocess
permissions = subprocess.run(['chmod','a+x','X-ray_Absorbance/install_xraylib.sh'])
%env PYTHONPATH=
!bash X-ray_Absorbance/install_xraylib.sh
import sys
_ = sys.path.append("/usr/local/lib/python3.7/site-packages")
import xraylib as xr
import math
import numpy as np
import matplotlib.pyplot as plt




In [48]:
#@title Now define a few functions.
#First - define a dictionary linking index to line descriptions
#This isn't used at the moment, but it could be used to output details about lines.
xr.XRayInit()

Klist = ['K']
Llist = ['L' + str(n) for n in range(1,4)]
Mlist = ['M' + str(n) for n in range(1,6)]
Nlist = ['N' + str(n) for n in range(1,8)]
Olist = ['O' + str(n) for n in range(1,8)]+['O45','O']
lineList = []
for k in Klist:
    for l in Llist:
        lineList.append(k+l+'_LINE')
    for m in Mlist:
        lineList.append(k+m+'_LINE')
    for n in Nlist:
        lineList.append(k+n+'_LINE')
    for o in Olist:
        lineList.append(k+o+'_LINE')
for l in Llist:
    for m in Mlist:
        lineList.append(l+m+'_LINE')
    for n in Nlist:
        lineList.append(l+n+'_LINE')
    for o in Olist:
        lineList.append(l+o+'_LINE')
for m in Mlist:
    for n in Nlist:
        lineList.append(m+n+'_LINE')
    for o in Olist:
        lineList.append(m+o+'_LINE')
for n in Nlist:
    for o in Olist:
        lineList.append(n+o+'_LINE')
fluorescenceLines = {}
for line in lineList:
    try:
        number = eval('xr.'+line)
        fluorescenceLines[number] = line.split('_LINE')[0]
    except:
        continue

#Now define our sliders
import ipywidgets as widgets
s1 = widgets.widgets.Text(
    value='H2O',
    placeholder='Enter chemical formula, e.g. H2 O',
    description='Sample:',
    disabled=False)
Excitation = widgets.FloatSlider(description = 'Excitation (keV)', 
    min = 5.,
    max = 15.,
    step = 0.01,
    value = 10.,
    layout=widgets.Layout(width='500px'))
Width = widgets.FloatText(description = 'Width (eV)', value = 190)
Range = widgets.FloatRangeSlider(description = 'Plot range (keV)', 
    min = 0.,
    max = 25.,
    step = 0.1,
    value = (5,15))
def update_excitation_range(*args):
  Excitation.min, Excitation.max = Range.value
Range.observe(update_excitation_range,'value')
ui = widgets.VBox([s1, widgets.HBox([Range,Width]), Excitation])

def Gauss(c,w,x):
    return np.exp(-((x-c)/w)**2/2)/(w*np.sqrt(2*np.pi))

def plotEmission(width,emissionDict,sample,xv,excit):
    total = np.zeros(len(xv)) #At the moment, this isn't used
    for Z,frac in zip(sample['Elements'],sample['massFractions']):
        symbol = xr.AtomicNumberToSymbol(Z)
        elemTotal = np.zeros(len(xv))
        for em in emissionDict:
            if em.split('-')[0] == symbol:
                elemTotal += Gauss(emissionDict[em][0],width,xv)*emissionDict[em][1]*frac
        plt.plot(xv,elemTotal, label = symbol)
        total += elemTotal
    # plt.plot(xv,total, 'k', label = 'total')
    axes = plt.gca()
    yLims = axes.get_ylim()
    plt.plot(xv,Gauss(excit,width,xv)*yLims[1]*width*np.sqrt(2*np.pi), 
             color = 'Black', label = 'Excitation')
    plt.legend()
    plt.xlabel('Energy (keV)')
    plt.ylabel('Relative intensity')
    plt.show()
    return
    
def calcEmission(E,sample):
    emission = {}
    for Z in sample['Elements']:
        symbol = xr.AtomicNumberToSymbol(Z)
        for line in range(1,384):
            try:
                emission_energy = xr.LineEnergy(Z,-line)
                if E > emission_energy:
                    # print(E,emission_energy,Z)
                    emission_intensity = xr.CS_FluorLine(Z,-line,E)
                    emission[symbol + '-' + fluorescenceLines[-line]] = \
                     [emission_energy, emission_intensity]
            except:
                continue
    return emission

def makePlot(s1,Excitation,Range,Width):
  try:
    p_sample = xr.CompoundParser(s1.replace(" ",""))
    xv = np.arange(Range[0],Range[1],(Range[1]-Range[0])/1000.)
    plotEmission(Width/1000.,calcEmission(Excitation,p_sample),p_sample,xv,Excitation)
  except:
    print('Enter valid sample - e.g. C2H6O')
  return

In [49]:
#@title Enter sample details 

#@markdown Enter composition as C2 H6 O for ethanol<br> 
#@markdown Excitation and plot energies are in keV; detector resolution in eV<br>
#@markdown Elastic scatter is scaled to the plot maximum - actual values will depend on concentration
out = widgets.interactive_output(makePlot, 
                           {'s1': s1, 'Excitation': Excitation, 'Range': Range, 'Width': Width})
display(ui,out)

Output()